In [4]:
import gtfs_kit as gk
import os.path
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import datetime as dt

In [5]:
gtfs_dir = "/mnt/c/Users/maita.schade/Nextcloud/Documents/Work/Gap_Map/raw/gtfs"
years = [2020, 2021]
paths = {y: os.path.join(gtfs_dir,str(y),str(y)+".zip") for y in years}
#feeds = {y: gk.read_feed(paths[y], dist_units='km') for y in years}

# TIME.

need dates to cut by...

In [4]:
window = ('20200301','20200331')

In [211]:
gtf = gk.read_feed(paths[2020], dist_units='km')

/home/mschade/anaconda3/envs/gapmapbox/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [212]:
gtf.stops.columns

Index(['stop_name', 'parent_station', 'stop_code', 'zone_id', 'stop_id',
       'stop_lat', 'stop_lon', 'location_type', 'wheelchair_boarding',
       'platform_code'],
      dtype='object')

In [6]:
gtf.calendar_dates.date[1] < gtf.calendar.start_date[1]

False

In [7]:
gtf.calendar.start_date[1]

'20191215'

In [8]:
gtf.calendar_dates.date[1]

'20191218'

In [9]:
type(gtf.calendar_dates.merge(gtf.calendar, on="service_id", how='outer').loc[272249].exception_type)

numpy.float64

Calculate the service windows

THE FOLLOWING IS A BIT USELESS--COMBINING `CALENDAR` AND `CALENDAR_DATES` UNNECESSARY

In [10]:
def first(s):
    s[0]

def safeMax(s):
    if all(pd.isna(s)):
        return pd.NA
    else:
        return s[~pd.isna(s)].max()
    
def safeMin(s):
    if all(pd.isna(s)):
        return pd.NA
    else:
        return s[~pd.isna(s)].min()
    

In [11]:
service_windows = gtf.calendar_dates.merge(gtf.calendar, on="service_id", how='outer').groupby("service_id").agg(
            min_exc=('date', safeMin),
            max_exc=('date', safeMax),
            start_date = ("start_date", 'first'),
            end_date = ("end_date", 'first')
    ).reset_index()

In [12]:
service_windows

,service_id,min_exc,max_exc,start_date,end_date
0,1,20191216,20201118,20200106,20201209
1,10,20191216,20201118,20200203,20201211
2,100,20200225,20201030,20200908,20200911
3,1000,20191220,20200611,20191216,20201211
4,10000,20200224,20200611,20200102,20200612
...,...,...,...,...,...
24767,9995,<NA>,<NA>,20200829,20201212
24768,9996,20191216,20200214,20200106,20200228
24769,9997,20191225,20200101,20191225,20200328
24770,9998,20200619,20201004,20200615,20201212


In [13]:
service_windows['first'] = service_windows.loc[:,'min_exc':'end_date'].apply(safeMin, axis=1) 

service_windows['last'] = service_windows.loc[:,'min_exc':'end_date'].apply(safeMax, axis=1)

service_windows = service_windows[['service_id','first','last']]

I don't think it actually makes sense to treat calendar and service together... what happens to an exception on an off day?
Unfortunately, in the constellation we have here, every exception is supposed to belong to a specific service referenced in calendar

In [14]:
gtf.calendar.service_id.isin(gtf.calendar_dates.service_id)

0         True
1        False
2         True
3         True
4         True
         ...  
23732    False
23733    False
23734    False
23735    False
23736    False
Name: service_id, Length: 23737, dtype: bool

In [15]:
(~gtf.calendar_dates.service_id.isin(gtf.calendar_dates.service_id)).sum()

0

Pick applicable services

In [16]:
services = service_windows[
    (service_windows['first'] < window[1]) & (service_windows['last'] > window[0])
    ].service_id

Filter calendar, calendar_dates by service

In [17]:
calendar = gtf.calendar[gtf.calendar.service_id.isin(services)]

In [18]:
calendar_dates = gtf.calendar_dates[gtf.calendar_dates.service_id.isin(services)]

Adjust dates so max, min are ok

First, the calendar itself

In [19]:
calendar.loc[23716,:]

monday               1
tuesday              1
wednesday            1
thursday             1
friday               1
saturday             1
sunday               1
start_date    20201109
end_date      20201212
service_id        4142
Name: 23716, dtype: object

In [20]:
calendar_dates

,service_id,exception_type,date
0,1,1,20191216
1,1,1,20191218
2,1,2,20200210
3,1,2,20200212
4,1,2,20200217
...,...,...,...
267890,9999,2,20200412
267891,9999,2,20200430
267892,9999,2,20200520
267893,9999,2,20200531


In [21]:
def elementMax(s, e):
    return s.apply(lambda e1: max(e1, e))

In [22]:
elementMax(calendar.start_date, window[0])

0        20200301
2        20200301
7        20200301
9        20200301
10       20200301
           ...   
23633    20200925
23676    20201101
23678    20201101
23705    20201107
23716    20201109
Name: start_date, Length: 14555, dtype: object

I think we need to...
- filter calendar services by window
- filter calendar_dates exceptions by window
- for each orphaned exception of type 1
    - create new calendar service of same ID and drop first exception
    - if there is more than one, leave it as exception

- filter calendar services by window

In [23]:
services = gtf.calendar[(gtf.calendar.start_date < window[1]) & (gtf.calendar.end_date > window[0])]

- filter calendar_dates exceptions by window

In [24]:
exceptions = gtf.calendar_dates[(gtf.calendar_dates.date < window[1]) & (gtf.calendar_dates.date > window[0])]

- for each orphaned exception of type 1

In [25]:
exceptions = exceptions.sort_values(by=["service_id", "date"])
orphans = exceptions[(~exceptions.service_id.isin(services.service_id)) & # is not in new service calendar
                     (exceptions.exception_type == 1)                     # and is ON
                    ]

first_orphans = orphans.groupby("service_id").first().reset_index()

    - create new calendar service of same ID

In [26]:
def serviceFromException(r):
    s = pd.Series( # create a service that has just one date, and is generally off
        {"monday":0,
         "tuesday":0,
         "wednesday":0,
         "thursday":0,
         "friday":0,
         "saturday":0,
         "sunday":0,
         "start_date": r.date,
         "end_date": r.date,
         "service_id": r.service_id
        }) 
    d = dt.datetime.strptime(r.date, "%Y%m%d").weekday()
    s[d] = r.exception_type # set the weekday of this exception's date to its type
    return(s)

In [27]:
services_expanded = services.append(
    first_orphans.apply(serviceFromException, axis=1)
)

    - drop first exception from exceptions
    - if there is more than one, leave additional ones as exception

In [28]:
# find which ones are in both, and keep only the rest
merge = exceptions.merge(first_orphans[['service_id', 'date']], how='outer', indicator='source') 
exceptions_reduced = merge[merge.source.eq('left_only')].drop('source', axis=1)

Now, we ought to filter the rest of the feed.
- `trips` by `service_id`
- `routes` by `route_id`
- `stop_times` by `trip_id`
- `stops` by `stop_id`
- `agency` by `agency_id`
- `transfers` by `stop_id`

First, check that we actually have a list of all services in `services_expanded` now

In [29]:
sum(~exceptions.service_id.isin(services.service_id))

2450

In [30]:
sum(~exceptions_reduced.service_id.isin(services_expanded.service_id))

0

- `trips` by `service_id`

In [31]:
trips = gtf.trips[gtf.trips.service_id.isin(services_expanded.service_id)]

In [32]:
(len(gtf.trips)-len(trips))/len(gtf.trips)

0.4146328832572849

In [33]:
len(trips)

1130348

- `routes` by `route_id`

In [34]:
routes = gtf.routes[gtf.routes.route_id.isin(trips.route_id)]

In [35]:
(len(gtf.routes)-len(routes))/len(gtf.routes)

0.19689716312056738

- `stop_times` by `trip_id`

In [36]:
stop_times = gtf.stop_times[gtf.stop_times.trip_id.isin(trips.trip_id)]

In [37]:
(len(gtf.stop_times)-len(stop_times))/len(gtf.stop_times)

0.4194993363971053

- `stops` by `stop_id`
    - here it's important to keep their parent stations, also!

In [38]:
stops = gtf.stops[gtf.stops.stop_id.isin(stop_times.stop_id)]

stops = stops.append(gtf.stops[gtf.stops.stop_id.isin(stops.parent_station)])

In [39]:
(len(gtf.stops)-len(stops))/len(gtf.stops)

0.17743325319586997

- `agency` by `agency_id` 

In [40]:
agency = gtf.agency[gtf.agency.agency_id.isin(routes.agency_id)]

In [41]:
(len(gtf.agency)-len(agency))/len(gtf.agency)

0.2635135135135135

- `transfers` by `stop_id`

In [42]:
sum((gtf.transfers.from_stop_id.isin(gtf.stops.stop_id)) & (gtf.transfers.to_stop_id.isin(gtf.stops.stop_id)))

102197

In [43]:
len(gtf.transfers)

102197

In [44]:
transfers = gtf.transfers[(gtf.transfers.from_stop_id.isin(stops.stop_id)) & (gtf.transfers.to_stop_id.isin(stops.stop_id))]

In [45]:
(len(gtf.transfers)-len(transfers))/len(gtf.transfers)

0.9995107488478135

In [46]:
len(transfers)

50

And new feed_info

In [47]:
feed_info = pd.DataFrame({
    'feed_publisher_name' : 'Agora Verkehrswende', 
    'feed_publisher_url' : 'https://www.agora-verkehrswende.de/',
    'feed_lang' : 'de',
    'feed_start_date' : window[0],
    'feed_end_date' : window[1], 
    'feed_version' : gtf.feed_info.feed_version + "_cut-"+window[0]+"--"+window[1],
    'feed_contact_email' : "maita.schade@agora-verkehrswende.de", 
    'feed_contact_url' : 'https://www.agora-verkehrswende.de/ueber-uns/team/maita/schade/'
    }, 
    index=[0])

In [48]:
feed_info

,feed_publisher_name,feed_publisher_url,feed_lang,feed_start_date,feed_end_date,feed_version,feed_contact_email,feed_contact_url
0,Agora Verkehrswende,https://www.agora-verkehrswende.de/,de,20200301,20200331,full-2020_cut-20200301--20200331,maita.schade@agora-verkehrswende.de,https://www.agora-verkehrswende.de/ueber-uns/t...


In [49]:
gtf_cut = gk.feed.Feed(dist_units=gtf.dist_units,
                       agency=agency,
                       stops=stops,
                       routes=routes,
                       trips=trips,
                       stop_times=stop_times,
                       calendar=services_expanded,
                       calendar_dates=exceptions_reduced,
                       transfers=transfers,
                       feed_info=feed_info
                      )

In [50]:
gtf_cut.validate()

,type,message,table,rows
0,warning,Feed expired,calendar/calendar_dates,[]
1,warning,Unrecognized column feed_contact_email,feed_info,[]
2,warning,Unrecognized column feed_contact_url,feed_info,[]
3,warning,"Repeated pair (route_short_name, route_long_name)",routes,"[1179, 1183, 1185, 1187, 1190, 1192, 1194, 119..."
5,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[31090762, 31090763, 31090764, 31090765, 31090..."
4,warning,Unrecognized column platform_code,stops,[]


In [51]:
gtf.validate()

,type,message,table,rows
3,error,Invalid route_type; maybe has extra space char...,routes,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
0,warning,Feed expired,calendar/calendar_dates,[]
1,warning,Unrecognized column feed_contact_email,feed_info,[]
2,warning,Unrecognized column feed_contact_url,feed_info,[]
4,warning,"Repeated pair (route_short_name, route_long_name)",routes,"[1, 2, 18, 48, 65, 79, 80, 85, 114, 115, 130, ..."
5,warning,Route has no trips,routes,"[1022, 1279, 1642, 2053, 2089, 2111, 2164, 251..."
8,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[31090762, 31090763, 31090764, 31090765, 31090..."
6,warning,Unrecognized column platform_code,stops,[]
7,warning,Stop has no stop times,stops,"[3034, 3870, 4130, 4352, 4432, 4829, 4830, 685..."


# SPACE.

First, get a shape to practice with

In [154]:
kreise_gdf = gpd.read_file("/mnt/c/Users/maita.schade/Nextcloud/Documents/Work/Gap_Map/raw/geo/vg250-ew_12-31.utm32s.shape.ebenen/vg250-ew_ebenen_1231/VG250_KRS.shp"
                          ).to_crs("epsg:4326")

shape = kreise_gdf[kreise_gdf.GEN=="Berlin"]
cut_name = "Berlin"

Try cutting the stops

In [214]:
stops_gdf = gpd.GeoDataFrame(gtf.stops.copy(), 
                             geometry=gpd.points_from_xy(gtf.stops.stop_lon, gtf.stops.stop_lat),
                             crs="epsg:4326"
                            )

In [215]:
gtf.stops.columns

Index(['stop_name', 'parent_station', 'stop_code', 'zone_id', 'stop_id',
       'stop_lat', 'stop_lon', 'location_type', 'wheelchair_boarding',
       'platform_code'],
      dtype='object')

Can it be that creating a geodataframe motifies the original dataframe in place?

Note to self: it is good to copy a dataframe before making a gdf from it!

In [216]:
stops = pd.DataFrame(gpd.sjoin(stops_gdf, shape[["geometry"]], op="within"
                              )).drop(columns=['geometry', 'index_right'])

Make sure all parent stations are there... I think there are edge cases that get lost otherwise

In [217]:
lost_parent_stations = stops[~stops.parent_station.isna() & ~stops.parent_station.isin(stops.stop_id)].parent_station.unique()

In [218]:
stops = stops.append(gtf.stops[gtf.stops.stop_id.isin(lost_parent_stations)])

In [219]:
stops.columns

Index(['stop_name', 'parent_station', 'stop_code', 'zone_id', 'stop_id',
       'stop_lat', 'stop_lon', 'location_type', 'wheelchair_boarding',
       'platform_code'],
      dtype='object')

Now we have a reduced set of stops and should just be able to trickle our way through the other tables:
- `stop_times` by `stop_id`
- `trips` by `trip_id`
- `calendar` and `calendar_dates` by `service_id`
- `routes` by `route_id` 
- `agency` by `agency_id`
- `transfers` by `stop_id`

- `stop_times` by `stop_id`


In [183]:
stop_times = gtf.stop_times[gtf.stop_times.stop_id.isin(stops.stop_id)]

In [184]:
(len(gtf.stop_times)-len(stop_times))/len(gtf.stop_times)

0.7922373614682496

- `trips` by `trip_id`


In [185]:
trips = gtf.trips[gtf.trips.trip_id.isin(stop_times.trip_id)]

In [186]:
(len(gtf.trips)-len(trips))/len(gtf.trips)

0.8191746586107663

- `calendar` and `calendar_dates` by `service_id`


In [187]:
calendar = gtf.calendar[gtf.calendar.service_id.isin(trips.service_id)]

(len(gtf.calendar)-len(calendar))/len(gtf.calendar)

0.8560053924253276

In [188]:
calendar_dates = gtf.calendar_dates[gtf.calendar_dates.service_id.isin(trips.service_id)]

In [189]:
(len(gtf.calendar_dates)-len(calendar_dates))/len(gtf.calendar_dates)

0.937811455980888

- `routes` by `route_id` 


In [190]:
routes = gtf.routes[gtf.routes.route_id.isin(trips.route_id)]

(len(gtf.routes)-len(routes))/len(gtf.routes)

0.9704787234042553

- `agency` by `agency_id`

In [191]:
agency = gtf.agency[gtf.agency.agency_id.isin(routes.agency_id)]

(len(gtf.agency)-len(agency))/len(gtf.agency)

0.8986486486486487

- `transfers` by `stop_id`

In [192]:
transfers = gtf.transfers[(gtf.transfers.from_stop_id.isin(stops.stop_id)) & (gtf.transfers.to_stop_id.isin(stops.stop_id))]

(len(gtf.transfers)-len(transfers))/len(gtf.transfers)

0.5229214164799358

So many transfers are in Berlin!!

And new feed_info

In [193]:
feed_info = pd.DataFrame({
    'feed_publisher_name' : 'Agora Verkehrswende', 
    'feed_publisher_url' : 'https://www.agora-verkehrswende.de/',
    'feed_lang' : 'de',
    'feed_start_date' : gtf.feed_info.feed_start_date,
    'feed_end_date' : gtf.feed_info.feed_end_date, 
    'feed_version' : gtf.feed_info.feed_version + "_cut-"+cut_name,
    'feed_contact_email' : "maita.schade@agora-verkehrswende.de", 
    'feed_contact_url' : 'https://www.agora-verkehrswende.de/ueber-uns/team/maita/schade/'
    }, 
    index=[0])

In [194]:
gtf_cut = gk.feed.Feed(dist_units=gtf.dist_units,
                       agency=agency,
                       stops=stops,
                       routes=routes,
                       trips=trips,
                       stop_times=stop_times,
                       calendar=calendar,
                       calendar_dates=calendar_dates,
                       transfers=transfers,
                       feed_info=feed_info
                      )

In [195]:
gtf_cut.validate()

,type,message,table,rows
3,error,Invalid route_type; maybe has extra space char...,routes,"[4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17,..."
0,warning,Feed expired,calendar/calendar_dates,[]
1,warning,Unrecognized column feed_contact_email,feed_info,[]
2,warning,Unrecognized column feed_contact_url,feed_info,[]
4,warning,"Repeated pair (route_short_name, route_long_name)",routes,"[114, 148, 402, 424, 447, 468, 500, 526, 536, ..."
8,warning,"Repeated pair (trip_id, departure_time)",stop_times,"[8261502, 12005087, 8470822, 8470853, 15946788..."
5,warning,Unrecognized column platform_code,stops,[]
6,warning,Unrecognized column geometry,stops,[]
7,warning,Stop has no stop times,stops,"[65357, 555037, 555927, 555966, 556229]"


In [227]:
gtf_cut.stops.loc[[65357, 555037, 555927, 555966, 556229]]

,stop_name,parent_station,stop_code,zone_id,stop_id,stop_lat,stop_lon,location_type,wheelchair_boarding,platform_code,geometry
65357,"Berlin, Washingtonplatz/Hauptbahnhof",35740,NaN,NaN,415688,52.523968,13.370708,NaN,NaN,NaN,None
555037,S Mahlsdorf Bhf (Berlin),292187,NaN,NaN,359476,52.512120,13.611230,NaN,NaN,NaN,None
555927,S+U Alexanderplatz Bhf (Berlin),361316,NaN,NaN,489037,52.521510,13.411267,NaN,NaN,NaN,None
555966,S+U Berlin Hauptbahnhof (tief),508163,NaN,NaN,64474,52.525043,13.369386,NaN,NaN,NaN,None
556229,S+U Potsdamer Platz Bhf (Berlin),698350,NaN,NaN,336238,52.509340,13.376454,NaN,NaN,NaN,None


Looks like the seemingly abandoned stations show up in the transfers:

In [230]:
zombie_stations = gtf_cut.stops.loc[[65357, 555037, 555927, 555966, 556229],["parent_station","stop_id"]]

In [233]:
gtf.transfers[gtf.transfers.from_stop_id.isin(zombie_stations.stop_id) | 
              gtf.transfers.to_stop_id.isin(zombie_stations.stop_id)]

,from_stop_id,to_stop_id,transfer_type,min_transfer_time
2178,11562,336238,2,240.0
2179,11562,336238,2,240.0
37655,336238,11562,2,240.0
37656,336238,11562,2,240.0
37657,336238,336238,2,240.0
...,...,...,...,...
90004,663925,336238,2,240.0
90005,663925,336238,2,240.0
92703,681301,489037,2,180.0
92704,681301,489037,2,180.0


In [234]:
zombie_stations[zombie_stations.stop_id.isin(gtf.transfers.from_stop_id) & 
                zombie_stations.stop_id.isin(gtf.transfers.to_stop_id)
               ]

,parent_station,stop_id
65357,35740,415688
555037,292187,359476
555927,361316,489037
555966,508163,64474
556229,698350,336238


A number of trips stop at multiple stops at the same time?!

In [224]:
pd.concat(g for _, g in gtf_cut.stop_times.groupby(["trip_id","departure_time"]) if len(g) > 1)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type
8261501,100002,12:48:00,12:48:00,116779,0,NaN,NaN,NaN
8261502,100002,12:48:00,12:48:00,319858,1,NaN,NaN,NaN
12005086,100003,11:25:00,11:25:00,638475,10,Fahrt endet hier,NaN,NaN
12005087,100003,11:25:00,11:25:00,578739,11,Fahrt endet hier,NaN,NaN
8470821,1000061,14:48:00,14:48:00,196543,0,U Berliner Str.,NaN,NaN
...,...,...,...,...,...,...,...,...
32762325,999946,05:03:00,05:03:00,456236,19,U Theodor-Heuss-Platz,NaN,NaN
32762342,999946,05:21:00,05:21:00,265480,36,U Theodor-Heuss-Platz,NaN,NaN
32762343,999946,05:21:00,05:21:00,224209,37,U Theodor-Heuss-Platz,NaN,NaN
36554192,999998,15:26:00,15:26:00,373804,34,NaN,NaN,NaN


Routes look like they're duped because they're in different agencies.

In [223]:
pd.concat(g for _, g in gtf_cut.routes.groupby(["route_short_name","route_long_name"]) if len(g) > 1)

,route_long_name,route_short_name,agency_id,route_type,route_id
2188,12,12,134,3,11853
2194,12,12,157,0,13655
2203,12,12,195,2,3597
3270,16,16,134,3,11689
3274,16,16,157,0,4107
6214,27,27,157,0,1474
6221,27,27,195,2,7783
12383,60,60,157,0,15899
12384,60,60,157,3,13214
12611,61,61,157,0,9845


Brosi has weird route_types!!

In [196]:
gtf_cut.routes.route_type.unique()

array([ 700,  900,    3, 1000,  109,  400,    0,    2,    4,    1])

We have edge cases where parent stations get cut away!

In [198]:
gtf_cut.stops.loc[605233]

stop_name              Stahnsdorfer Brücke
parent_station                      166875
stop_code                          SDBR 01
zone_id                                NaN
stop_id                             138897
stop_lat                         52.391384
stop_lon                         13.131384
location_type                          NaN
wheelchair_boarding                    NaN
platform_code                          NaN
geometry                              None
Name: 605233, dtype: object

In [199]:
gtf_cut.stops[gtf_cut.stops.stop_id==166875]

,stop_name,parent_station,stop_code,zone_id,stop_id,stop_lat,stop_lon,location_type,wheelchair_boarding,platform_code,geometry


In [200]:
gtf.stops[gtf.stops.stop_id=='166875']

,stop_name,parent_station,stop_code,zone_id,stop_id,stop_lat,stop_lon,location_type,wheelchair_boarding,platform_code,geometry
64756,"Berlin, Stahnsdorfer Brücke",NaN,NaN,NaN,166875,52.391556,13.131558,1.0,NaN,NaN,POINT (13.13156 52.39156)


# Test:

In [1]:
import gtfs_kit_scissors as gks

In [6]:
gtf = gk.read_feed(paths[2021], dist_units='km')

In [7]:
kreise_gdf = gpd.read_file("/mnt/c/Users/maita.schade/Nextcloud/Documents/Work/Gap_Map/raw/geo/vg250-ew_12-31.utm32s.shape.ebenen/vg250-ew_ebenen_1231/VG250_KRS.shp"
                          ).to_crs("epsg:4326")

shape = kreise_gdf[kreise_gdf.GEN=="Essen"]

In [8]:
shape

,ADE,GF,BSG,ARS,AGS,SDV_ARS,GEN,BEZ,IBZ,BEM,...,ARS_0,AGS_0,WSK,EWZ,KFL,DEBKG_ID,RS,SDV_RS,RS_0,geometry
65,4,4,1,05113,05113,051130000000,Essen,Kreisfreie Stadt,40,--,...,051130000000,05113000,2006-10-01,582760,210.34,DEBKGDL20000E4ZN,05113,051130000000,051130000000,"POLYGON ((7.01895 51.52054, 7.02071 51.51806, ..."


In [9]:
mar21_scissored = gks.cut_dates(gtf, ["20210301","20210331"], validate=False)

Calendar reduction: 22.0%
Calendar_dates reduction: 97.6%
Trips reduction: 50.8%
Routes reduction: 14.7%
Stop_times reduction: 49.8%
Stops reduction: 12.5%
Agencies reduction: 11.7%


In [10]:
mar21_gtk = gtf.restrict_to_dates(["20210301","20210331"])

In [13]:
mar21_scissored.routes

,route_long_name,route_short_name,agency_id,route_type,route_id
0,0,0,252,0,7896
1,000,000,120,3,14654
3,001,001,120,3,6811
4,001,001,4,3,19387
5,002,002,120,3,17729
...,...,...,...,...,...
23758,n65,n65,97,3,6648
23760,n71,n71,97,3,16393
23761,n72,n72,97,3,13830
23763,n96,n96,97,3,13144


In [11]:
mar21_gtk.routes

,route_long_name,route_short_name,agency_id,route_type,route_id
0,0,0,252,0,7896
1,000,000,120,3,14654
3,001,001,120,3,6811
4,001,001,4,3,19387
5,002,002,120,3,17729
...,...,...,...,...,...
23758,n65,n65,97,3,6648
23760,n71,n71,97,3,16393
23761,n72,n72,97,3,13830
23763,n96,n96,97,3,13144
